# Land Migrator

Migrator script to mint land nfts from the script

In [1]:
import csv
import io
import json
import uuid
import pandas as pd
import numpy as np

In [2]:
import os; os.getcwd()

'/Users/saz/development/near/NearZero/migrators/LandMigrator/src'

In [3]:
from dotenv import dotenv_values
config = dotenv_values()

In [5]:
df = pd.read_csv('../data/factors_influencing_organic_carbon_accumulation_TOC_SAR.csv')
df.head()

# Fix dataframe column headers
df['TOC'] = df['TOC\n(g m-2 yr-1)']
df['SAR'] = df['SAR\n (mm yr-1)']
df['Sedimentary Environment'] = df['Sedimentary \nenvironment']
df['Environmental Condition'] = df['Environmental\nCondition']
df = df.drop(['TOC\n(g m-2 yr-1)', 'SAR\n (mm yr-1)', 'Sedimentary \nenvironment', 'Environmental\nCondition'], axis=1)
df.head()

,Region,Location,Weather,Mangrove species,Source,TOC,SAR,Sedimentary Environment,Environmental Condition
0,NORTH AMERICA (n = 22),"Naples Bay, Florida - USA",Subtropical,Avicennia germinans\nLaguncularia racemona\nRh...,"Marchio et al., 2016",50,1.7,Adjacent sites,Conserved
1,NORTH AMERICA (n = 22),"Naples Bay, Florida - USA",Subtropical,Avicennia germinans\nLaguncularia racemona\nRh...,"Marchio et al., 2016",74,1.6,Adjacent sites,Degraded by urban development
2,NORTH AMERICA (n = 22),"Florida Keys, Southern Florida - USA",Tropical,Avicennia germinans,"Callaway et al., 1997",115,1.9,Adjacent sites,Impacted by storms
3,NORTH AMERICA (n = 22),"Florida Keys, Southern Florida - USA",Tropical,Avicennia germinans,"Callaway et al., 1997",156,1.9,Adjacent sites,Impacted by storms
4,NORTH AMERICA (n = 22),"Rookery Bay, Southwest Florida - USA",Temperate,Avicennia germinans\nRhizophora mangle,"Lynch, 1989",90,1.7,Adjacent sites,Impacted by frequent flooding


# Run to make queries

In [8]:
# For now size is random bc I don't have a proper value but in the future we should determine size
import random # for generating land size
def get_land_size():
    """
    land size should be provided but it's not so just giving a value with a bit of jitter
    """
    base = 60_000_000
    variance = random.randint(-100, 100) * 10_000 
    return base + variance


def _get_description(row)->str:
    weather = row['Weather']
    env_cond = row['Environmental Condition']
    return f"{weather}, {env_cond}, {row['Location']}"
    
    
def make_metadata(owner_id: str, row):
    '''
    Create the metadata json to send to nft.storage
    '''
    token_id = str(uuid.uuid4())
    title = f"Land#{token_id.split('-')[-1]}"

    properties = {
        **row,
        "name": title,
        "description": _get_description(row),
        "image": "https://bafkreihdakgwl2lxgwejyrcvdzeyhtxex7tspfabngamyj3vbznyu2snjm.ipfs.nftstorage.link/"
    }
    
    # Transform to conform to standard
    properties = { k: {
        "description": v,
        "type": 'string' if type(v) is str else 'integer'
    } for k,v in properties.items()}
    
    metadata = {
        "token_id": token_id,
        "title": title,
        "type": "object",
        "properties": properties,
    }
    return metadata

    
def make_mint_data(owner_id: str, metadata, storage_url) -> str:
    properties = metadata['properties']
    mint_data = {
        "token_id": metadata['token_id'],
        "receiver_id": owner_id,
        "metadata":
        {
            "title": metadata['title'],
            "description": properties['description']['description'],
            "media": properties['image']['description'],
            "reference": storage_url,
            "land_toc": properties['TOC']['description'],
            "land_size": get_land_size(),
            "copies": 1
        }
    }
    return mint_data

def store_metadata(metadata) -> str:
    metadata_json = json.dumps(metadata)
    stream = os.popen(f"node storage.js '{metadata_json}'")
    metadata_url = stream.read()
    stream.close()
    return metadata_url.rstrip('\n')

def mint(mint_data):
    mintdata_json = json.dumps(mint_data)
    cmd = \
        f"near call landnft_nearzero.testnet nft_mint '{mintdata_json}' --accountId nearzero.testnet --deposit 0.1"
    stream = os.popen(cmd)
    data = stream.read()
    print(data)
    stream.close()


def make_queries(row):
    owner = 'nearzero.testnet'
    metadata = make_metadata(owner, row)
    storage_url = store_metadata(metadata)
    mint_arguments = make_mint_data(owner, metadata, storage_url)
    return mint_arguments
    

# RUN ALL
# datadict = df.to_dict('index')
# TEST
datadict = {1: df.to_dict('index')[1]}

queries = [make_queries(row) for row in datadict.values()]
for query in queries:
    mint(query)
queries[:10]

Scheduling a call: landnft_nearzero.testnet.nft_mint({"token_id": "de9d94f1-029c-46a1-9417-901b21ced1fd", "receiver_id": "nearzero.testnet", "metadata": {"title": "Land#901b21ced1fd", "description": "Subtropical, Degraded by urban development, Naples Bay, Florida - USA", "media": "https://bafkreihdakgwl2lxgwejyrcvdzeyhtxex7tspfabngamyj3vbznyu2snjm.ipfs.nftstorage.link/", "reference": "https://bafkreibev2e6dwkjrbs5iavwinhqt7cmvvdnqmxoxfmljv5ixjs67hgviu.ipfs.nftstorage.link/", "land_toc": 74, "land_size": 60520000, "copies": 1}}) with attached 0.1 NEAR
Doing account.functionCall()
Receipts: H2csTVvvhkN2J4gCTTUouVNqbHR8vZ95uc2zixLf6j8K, B9LSC1KVMqYEKZFzPXgA1GUhHiirbcmm7K8mbwinF6Rg
	Log [landnft_nearzero.testnet]: EVENT_JSON:{"standard":"nep171","version":"nft-1.0.0","event":"nft_mint","data":[{"owner_id":"nearzero.testnet","token_ids":["de9d94f1-029c-46a1-9417-901b21ced1fd"]}]}
	Log [landnft_nearzero.testnet]: Refunding 89680000000000000000000 yoctoNEAR
Transaction Id 2Ecc28TTfzb1xRso2TA2

[{'token_id': 'de9d94f1-029c-46a1-9417-901b21ced1fd',
  'receiver_id': 'nearzero.testnet',
  'metadata': {'title': 'Land#901b21ced1fd',
   'description': 'Subtropical, Degraded by urban development, Naples Bay, Florida - USA',
   'media': 'https://bafkreihdakgwl2lxgwejyrcvdzeyhtxex7tspfabngamyj3vbznyu2snjm.ipfs.nftstorage.link/',
   'reference': 'https://bafkreibev2e6dwkjrbs5iavwinhqt7cmvvdnqmxoxfmljv5ixjs67hgviu.ipfs.nftstorage.link/',
   'land_toc': 74,
   'land_size': 60520000,
   'copies': 1}}]

# Setup Bundler

